Функция для конвертации модели в формат ONNX с помощью PyTorch

In [1]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)


def convert_from_torch_to_onnx(
        onnx_path: str,
        tokenizer: AutoTokenizer,
        model: AutoModelForSequenceClassification
) -> None:
    """Конвертация модели из формата PyTorch в формат ONNX.

    @param onnx_path: путь к модели в формате ONNX
    @param tokenizer: токенизатор
    @param model: модель
    """
    dummy_model_input = tokenizer(
        "текст для конвертации",
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    ).to("cpu")
    torch.onnx.export(
        model,
        dummy_model_input["input_ids"],
        onnx_path,
        opset_version=12,
        input_names=["input_ids"],
        output_names=["output"],
        dynamic_axes={
            "input_ids": {
                0: "batch_size",
                1: "sequence_len"
            },
            "output": {
                0: "batch_size"
            }
        }
    )

Функция для квантизации модели в формате ONNX

In [3]:
from onnxruntime.quantization import (
    quantize_dynamic,
    QuantType
)


def convert_from_onnx_to_quantized_onnx(
        onnx_model_path: str,
        quantized_onnx_model_path: str
) -> None:
    """Квантизация модели в формате ONNX до Int8
    и сохранение кванитизированной модели на диск.

    @param onnx_model_path: путь к модели в формате ONNX
    @param quantized_onnx_model_path: путь к квантизированной модели
    """
    quantize_dynamic(
        onnx_model_path,
        quantized_onnx_model_path,
        weight_type=QuantType.QUInt8
    )

Функция для инференса модели c с помощью PyTorch

In [4]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)


torch.set_num_threads(1)


def pytorch_inference(
        text: str,
        max_tokens: int,
        model: AutoModelForSequenceClassification,
        tokenizer: AutoTokenizer,
) -> torch.Tensor:
    """Инференс модели с помощью PyTorch.

    @param text: входной текст для классификации
    @param max_tokens: максимальная длина последовательности в токенах
    @param model: BERT-модель
    @param tokenizer: токенизатор
    @return: логиты на выходе из модели
    """
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_tokens,
        return_tensors="pt"
    ).to('cpu')
    with torch.inference_mode():
        outputs = model(**inputs).logits.detach()
    return outputs
  

Функция для создания сессии ONNX Runtime

In [5]:
import onnxruntime
from onnxruntime import (
    InferenceSession,
    SessionOptions
)


def create_onnx_session(
        model_path: str,
        provider: str = "CPUExecutionProvider"
) -> InferenceSession:
    """Создание сессии для инференса модели с помощью ONNX Runtime.

    @param model_path: путь к модели в формате ONNX
    @param provider: инференс на ЦП
    @return: ONNX Runtime-сессия
    """
    options = SessionOptions()
    options.graph_optimization_level = \
        onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
    options.intra_op_num_threads = 1
    session = InferenceSession(model_path, options, providers=[provider])
    session.disable_fallback()
    return session


Функция для инференса модели c помощью ONNX Runtime

In [6]:
import numpy as np
from transformers import AutoTokenizer
from onnxruntime import InferenceSession


def onnx_inference(
        text: str,
        session: InferenceSession,
        tokenizer: AutoTokenizer,
        max_length: int
) -> np.ndarray:
    """Инференс модели с помощью ONNX Runtime.

    @param text: входной текст для классификации
    @param session: ONNX Runtime-сессия
    @param tokenizer: токенизатор
    @param max_length: максимальная длина последовательности в токенах
    @return: логиты на выходе из модели
    """
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="np",
    )
    input_feed = {
        "input_ids": inputs["input_ids"].astype(np.int64)
    }
    outputs = session.run(
        output_names=["output"],
        input_feed=input_feed
    )[0]
    return outputs
